## Importamos las librerias necesarias

In [2]:
#Importamos pandas, numpy
import numpy as np
import pandas as pd

## Importamos el dataset limpio para cargarlo en un dataframe

In [3]:
#Guardamos el path del archivo en una variable para despues cargarlo en un df

file = 'C:/Users/Ramiro/Desktop/Digital House/Desafios/Desafio 2/dataset limpio.csv'

# Creamos un DataFrame con la info del csv
df = pd.read_csv(file)

### Dejamos solo las columnas necesarias

In [4]:
df = df[['Tipo_Propiedad', 'Barrio','Provincia', 'Precio', 'Precio_ARS', 'Precio_Dolares', 'Metros_Totales',
       'Metros_Cubiertos', 'Precio_m2_Dls', 'Precio_m2', 'Cantidad_Ambientes']]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121220 entries, 0 to 121219
Data columns (total 11 columns):
Tipo_Propiedad        121220 non-null object
Barrio                121197 non-null object
Provincia             121220 non-null object
Precio                100810 non-null float64
Precio_ARS            97850 non-null float64
Precio_Dolares        97389 non-null float64
Metros_Totales        108336 non-null float64
Metros_Cubiertos      108865 non-null float64
Precio_m2_Dls         67754 non-null float64
Precio_m2             87371 non-null float64
Cantidad_Ambientes    86078 non-null float64
dtypes: float64(8), object(3)
memory usage: 10.2+ MB


### Antes de eliminar observaciones, deberiamos chequear la distribucion por Provincias para asegurarnos que no nos queden zonas sin observaciones

In [6]:
df['Provincia'].value_counts()

Capital Federal                 32316
Bs.As. G.B.A. Zona Norte        25560
Bs.As. G.B.A. Zona Sur          13952
Córdoba                         12069
Santa Fe                        10172
Buenos Aires Costa Atlántica    10006
Bs.As. G.B.A. Zona Oeste         9322
Buenos Aires Interior            2291
Río Negro                         808
Neuquén                           733
Mendoza                           681
Tucumán                           674
Corrientes                        583
Misiones                          464
Entre Ríos                        369
Salta                             278
Chubut                            259
San Luis                          252
La Pampa                          157
Formosa                            65
Chaco                              57
San Juan                           40
Tierra Del Fuego                   31
Catamarca                          27
Jujuy                              26
Santa Cruz                         20
Santiago Del

Vemos que en el dataset original existen provincias con muy pocas observaciones. Esto generara que el modelo no pueda predecir correctamente, o del todo, esas provincias.

### Nos quedamos solo con las observaciones que no tienen valores nulos

Prodiamos intentar predecir cualquiera de las variables de precio (en dolares o pesos). Para evaluar que tan bien performan los modelos, vamos a intentar predecir tanto el Precio total en pesos (Precio_ARS) y el Precio por metro cuadrado en pesos (Precio_m2)

In [5]:
df_precio = df.drop(['Precio', 'Precio_Dolares','Precio_m2_Dls', 'Precio_m2'], axis = 1)

df_m2 = df.drop(['Precio', 'Precio_Dolares','Precio_m2_Dls', 'Precio_ARS'], axis = 1)

In [6]:
df_precio = df_precio.dropna(axis =0 , how = 'any')
df_m2 = df_m2.dropna(axis =0 , how = 'any')

In [9]:
#Dataset Precio total
df_precio['Provincia'].value_counts()

Capital Federal                 21400
Bs.As. G.B.A. Zona Norte        15078
Buenos Aires Costa Atlántica     7242
Bs.As. G.B.A. Zona Sur           6450
Bs.As. G.B.A. Zona Oeste         5315
Córdoba                          4606
Santa Fe                         2874
Buenos Aires Interior            1098
Corrientes                        308
Mendoza                           288
Río Negro                         257
Neuquén                           223
Misiones                          216
Tucumán                           103
Chubut                             97
San Luis                           79
Entre Ríos                         58
Salta                              57
Tierra Del Fuego                   24
Chaco                              16
Santa Cruz                         10
La Pampa                            9
Jujuy                               6
Catamarca                           4
San Juan                            3
La Rioja                            2
Santiago Del

In [10]:
#Dataset Precio m2
df_m2['Provincia'].value_counts()

Capital Federal                 21418
Bs.As. G.B.A. Zona Norte        15065
Buenos Aires Costa Atlántica     6208
Bs.As. G.B.A. Zona Sur           6086
Bs.As. G.B.A. Zona Oeste         5094
Córdoba                          4428
Santa Fe                         2711
Buenos Aires Interior            1042
Corrientes                        288
Mendoza                           285
Neuquén                           209
Río Negro                         176
Misiones                          165
San Luis                           76
Tucumán                            65
Salta                              53
Entre Ríos                         53
Chubut                             32
Tierra Del Fuego                   24
Chaco                              15
La Pampa                            9
Catamarca                           7
Santa Cruz                          7
Jujuy                               4
San Juan                            3
La Rioja                            2
Santiago Del

Efectivamente, notamos que al momento de quedarnos solo con las observaciones con todos los valores completos, perdimos Provincias enteras como Formosa, Catamarca y Jujuy.
Esto va a causar que al momento de querer realizar una prediccion sobre una propiedad de esa provincia, el modelo prediga incorrectamente.
Entendemos que es un error a tener en cuenta. La forma de poder corregirlo seria agregar observaciones al dataset.

## Modelado

Para comenzar el modelado, debemos transformar las variables categoricas en dummies, luego normalizaremos las variables para facilitar la corrida de los distintos modelos.
Vamos a modelar primero tratando predecir el Precio total para todo el pais, y luego el precio por metro cuadrado.

Finalmente, generaremos modelos mas acotados (a ciertas provincias o zonas) buscando una performance mejor de los modelos

In [7]:
#Creamos variables dummies de las variables categoricas

#Provincia
df_precio = pd.concat([df_precio, pd.get_dummies(data = df_precio['Provincia'], prefix= 'prov')], axis = 1)
df_m2 = pd.concat([df_m2, pd.get_dummies(data = df_m2['Provincia'], prefix= 'prov')], axis = 1)

#Barrio
df_precio = pd.concat([df_precio, pd.get_dummies(data = df_precio['Barrio'], prefix= 'barrio')], axis =1)
df_m2 = pd.concat([df_m2, pd.get_dummies(data = df_m2['Barrio'], prefix= 'barrio')], axis = 1)

#Tipo Propiedad
df_precio = pd.concat([df_precio, pd.get_dummies(data = df_precio['Tipo_Propiedad'], prefix= 'tipo_prop')], axis =1)
df_m2 = pd.concat([df_m2, pd.get_dummies(data = df_m2['Tipo_Propiedad'], prefix= 'tipo_prop')], axis = 1)

In [8]:
#Dropeo de las columnas originales
df_precio = df_precio.drop(['Provincia','Barrio','Tipo_Propiedad'], axis = 1)
df_m2 = df_m2.drop(['Provincia','Barrio','Tipo_Propiedad'], axis = 1)

#### Importamos todas las librerias necesarias para el modelado

In [9]:
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

#### Armamos nuestro vector objetivo y nuestra matriz de features

In [10]:
#Separamos nuestra matriz de features

X = df_precio.drop(['Precio_ARS'], axis = 1)
y = df_precio['Precio_ARS']

#### Separamos los datos en sets de testeo y training

In [11]:
#Separamos en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1)

### Regresion Lineal

In [12]:
#Instanciamos el modelo

lm = linear_model.LinearRegression()

#Entrenamos el modelo
reg_lineal = lm.fit(X_train, y_train)

#Guardamos el score del modelo en una variable para poder comparar
R2_reg_lin_precio = reg_lineal.score(X_test, y_test)

#Prediccion con el set de testeo

print('Score de Regresion Lineal:', reg_lineal.score(X_test, y_test))

Score del Regresion Lineal: 0.568574891331734


#### Como predice el mismo modelo el precio por metro cuadrado?

In [16]:
#Separamos nuestra matriz de features

X_m2 = df_m2.drop(['Precio_m2'], axis = 1)
y_m2 = df_m2['Precio_m2']

In [17]:
#Separamos en train y test
X_train_m2, X_test_m2, y_train_m2, y_test_m2 = train_test_split(X_m2, y_m2, test_size=0.10, random_state=15)

In [18]:
#Instanciamos el modelo

lm_m2 = linear_model.LinearRegression()

#Entrenamos el modelo
reg_lineal_m2 = lm_m2.fit(X_train_m2, y_train_m2)

#Guardamos el score del modelo en una variable para poder comparar
R2_reg_lin_m2 = reg_lineal_m2.score(X_test_m2, y_test_m2)

#Prediccion con el set de testeo

print('Score de Regresion Lineal:', reg_lineal_m2.score(X_test_m2, y_test_m2))

Score de Regresion Lineal: 0.22195752473918462


### Comparemos los resultados de los dos modelos

In [45]:
print('R2 Modelo Precio: ',R2_reg_lin_precio)
print('Ordenada al origen Modelo Precio: ',reg_lineal.intercept_)
print('R2 Modelo Precio_m2: ', R2_reg_lin_m2)
print('Ordenada al origen Modelo m2: ',reg_lineal_m2.intercept_)

R2 Modelo Precio:  0.568574891331734
Ordenada al origen Modelo Precio:  -412099.3669661698
R2 Modelo Precio_m2:  0.22195752473918462
Ordenada al origen Modelo Precio:  11808.73003027371


In [58]:
#Creamos un diccionario con la informacion para luego amar un dataframe y comparar los modelos

rl_argentina_precio = { 'Modelo': "Regresion Lineal",
                        'Alcance del modelo': 'Nacional',
                        'Variable Obejtivo': 'Precio_ARS',
                        'R2': R2_reg_lin_precio,
                        'Intercepto': reg_lineal.intercept_}

rl_argentina_m2 = { 'Modelo': "Regresion Lineal",
                    'Alcance del modelo': 'Nacional',
                    'Variable Obejtivo': 'Precio_m2',
                    'R2': R2_reg_lin_m2,
                    'Intercepto': reg_lineal_m2.intercept_}

# Xq la ordenada al origen da negativa??

Vemos que el modelo performa mejor cuando intentamos predecir el precio total que el precio por metro cuadro.
Veamoslo graficamente

In [31]:
import plotly.plotly as py
import plotly.graph_objs as go

In [32]:
data = [go.Bar(
            x=['Precio_ARS', 'Precio_m2'],
            y=[R2_reg_lin_precio,R2_reg_lin_m2]
    )]

py.iplot(data, filename='basic-bar')

Vemos que hay una diferencia considerable entre el score del modelo que predice el precio total, que el que predice el precio por metro cuadrado. Tengamos en cuenta que en ninguno de los dos modelos incluimos la otra variable al momento de modelar, ya que entendemos que por ejemplo, el precio por metro cuadrado explicaria casi el total de la variabilidad del Precio Total de la propiedad

El modelo de regresion lineal multiple es demasiado simple para poder obtener un insight lo suficientemente convincente, por lo cual vamos a aplicar dos modelos que incluyen tecnicas de regularizacion: Ridge y Lasso

### Primero debemos normalizar las features.
Las variables dummies no es necesario que sean normalizadas ya que son variables dicotomicas (0 o 1).

Recordemos que nuestros sets de training y testeo quedaron definidos de la siguiente manera:

`X_train, X_test, y_train, y_test`: Para el modelar el Precio_ARS

`X_train_m2, X_test_m2, y_train_m2, y_test_m2`: Para modelar el Precio_m2

In [34]:
#Importamos StandardScaler para normalizar las variables
from sklearn.preprocessing import StandardScaler
scaler_precio = StandardScaler()
X_train = scaler_precio.fit_transform(X_train)

In [35]:
print('media:',np.mean(X_train[:,1]))
print('desvio:',np.std(X_train[:,1]))

media: -2.5427072006539488e-17
desvio: 1.0000000000000002


## Regresion Lineal Ridge

In [23]:
#Instanciamos el modelo

lm_ridge =  linear_model.RidgeCV(alphas=[0.1, 20, 100], cv=5)

#Entrenamos el modelo
reg_lineal_ridge = lm_ridge.fit(X_train, y_train)

# Lo utilizamos para predecir en test
X_test = scaler_precio.transform(X_test)

#Guardamos el score en una variable
score_ridge_precio = reg_lineal_ridge.score(X_test, y_test)

#Prediccion con el set de testeo

print('Score del modelo Ridge:', reg_lineal_ridge.score(X_test, y_test))

Score del modelo Ridge: 0.5693368190898047


#### Como predice el mismo modelo el precio por metro cuadrado?

In [24]:
#Normalizamos las features

scaler_m2 = StandardScaler()
X_train_m2 = scaler_m2.fit_transform(X_train_m2)

In [26]:
print('media:',np.mean(X_train_m2[:,1]))
print('desvio:',np.std(X_train_m2[:,1]))

media: -2.3861648901743697e-17
desvio: 0.9999999999999999


In [27]:
#Instanciamos el modelo
lm_ridge_m2 =  linear_model.RidgeCV(alphas=[0.1, 20, 10], cv=5)

#Entrenamos el modelo
reg_lineal_ridge_m2 = lm_ridge_m2.fit(X_train_m2, y_train_m2)

# Lo utilizamos para predecir en test
X_test_m2 = scaler_m2.transform(X_test_m2)

score_ridge_m2 = reg_lineal_ridge_m2.score(X_test_m2, y_test_m2)

#Prediccion con el set de testeo
print('Score del modelo Ridge:', reg_lineal_ridge_m2.score(X_test_m2, y_test_m2))

Score del modelo Ridge: 0.22245239063011887


### Comparemos los resultados de los dos modelos

In [42]:
print('Score Ridge Precio: ', score_ridge_precio)
print('Alpha Ridge Precio: ', reg_lineal_ridge.alpha_)
print('Ordenada al origen Precio: ', reg_lineal_ridge.intercept_)
print('Score Ridge m2: ', score_ridge_m2)
print('Alpha Ridge m2: ', reg_lineal_ridge_m2.alpha_)
print('Ordenada al origen m2: ', reg_lineal_ridge_m2.intercept_)

Score Ridge Precio:  0.5693368190898047
Alpha Ridge Precio:  20
Ordenada al origen Precio:  3471870.1225360385
Score Ridge m2:  0.22245239063011887
Alpha Ridge m2:  20
Ordenada al origen m2:  5665.297687313675


In [59]:
#Creamos un diccionario para luego mostrar los datos en un dataframe

ridge_argentina_precio = {'Modelo': "Regresion Lineal Ridge",
                          'Alcance del modelo': 'Nacional',
                          'Variable Obejtivo': 'Precio_ARS',
                          'R2': score_ridge_precio,
                          'Intercepto': reg_lineal_ridge.intercept_,
                          'Alpha del modelo': reg_lineal_ridge.alpha_}

ridge_argentina_m2 = { 'Modelo': "Regresion Lineal Ridge",
                       'Alcance del modelo': 'Nacional',                      
                       'Variable Obejtivo': 'Precio_m2',
                       'R2': score_ridge_m2,
                       'Intercepto': reg_lineal_ridge_m2.intercept_,
                       'Alpha del modelo': reg_lineal_ridge_m2.alpha_}

# COMO NOS CONVIENE GRAFICAR ESTOS DATOS???

## Regresion Lasso

In [51]:
#Instanciamos el modelo
lm_lasso =  linear_model.LassoCV(alphas=[1000, 100000, 1000], cv = 5)

#Entrenamos el modelo
reg_lineal_lasso = lm_lasso.fit(X_train, y_train)

#Guardamos el score
score_lasso_precio = reg_lineal_lasso.score(X_test, y_test)

print('Score del modelo Lasso:', reg_lineal_lasso.score(X_test, y_test))

Score del modelo Lasso: 0.571130490164745


### Como funciona el modelo para el precio por metro cuadrado?

In [52]:
#Instanciamos el modelo
lm_lasso_m2 =  linear_model.LassoCV(alphas=[10000, 100000, 1000], cv = 5)

#Entrenamos el modelo
reg_lineal_lasso_m2 = lm_lasso_m2.fit(X_train_m2, y_train_m2)

#Guardamos el score
score_lasso_m2 = reg_lineal_lasso_m2.score(X_test_m2, y_test_m2)

print('Score del modelo Lasso:', reg_lineal_lasso_m2.score(X_test_m2, y_test_m2))

Score del modelo Lasso: 0.1321714542257224


In [53]:
print('Score Lasso Precio: ', score_lasso_precio)
print('Ordenada al origen Lasso Precio: ', reg_lineal_lasso.intercept_)
print('Alpha Lasso Precio: ', reg_lineal_lasso.alpha_)
print('Score Lasso m2: ', score_lasso_m2)
print('Ordenada al origen Lasso m2: ', reg_lineal_lasso_m2.intercept_)
print('Alpha Lasso m2: ', reg_lineal_lasso_m2.alpha_)

Score Lasso Precio:  0.571130490164745
Ordenada al origen Lasso Precio:  3471870.1225360385
Alpha Lasso Precio:  1000
Score Lasso m2:  0.1321714542257224
Ordenada al origen Lasso Precio:  5665.297687313675
Alpha Lasso m2:  1000


In [60]:
#Creamos un diccionario para luego mostrar los datos en un dataframe

lasso_argentina_precio = {'Modelo': "Regresion Lineal Lasso",
                          'Alcance del modelo': 'Nacional',
                          'Variable Obejtivo': 'Precio_ARS',
                          'R2': score_lasso_precio,
                          'Intercepto': reg_lineal_lasso.intercept_,
                          'Alpha del modelo': reg_lineal_lasso.alpha_}

lasso_argentina_m2 = { 'Modelo': "Regresion Lineal Lasso",
                       'Alcance del modelo': 'Nacional',                      
                       'Variable Obejtivo': 'Precio_m2',
                       'R2': score_lasso_m2,
                       'Intercepto': reg_lineal_lasso_m2.intercept_,
                       'Alpha del modelo': reg_lineal_lasso_m2.alpha_}

## Elastic Net

In [54]:
#Instanciamos el modelo

lm_elastic_net =  linear_model.ElasticNetCV(l1_ratio= [.1, .5, .7, .9, .95, .99, 1],cv = 5)

#Entrenamos el modelo
elastic_net = lm_elastic_net.fit(X_train, y_train)

#Guardamos el score
score_EN = elastic_net.score(X_test, y_test)

#Prediccion con el set de testeo

print('Score del modelo Elastic Net:', elastic_net.score(X_test, y_test))

Score del modelo Elastic Net: 0.5717193307475611


### Como funciona el modelo para el precio por metro cuadrado?

In [55]:
#Instanciamos el modelo

lm_elastic_net_m2 =  linear_model.ElasticNetCV(l1_ratio= [.1, .5, .7, .9, .95, .99, 1], alphas = [10, 100, 10],cv = 5)

#Entrenamos el modelo
elastic_net_m2 = lm_elastic_net_m2.fit(X_train_m2, y_train_m2)

#Guardamos el score
score_EN_m2 = elastic_net_m2.score(X_test_m2, y_test_m2)

#Prediccion con el set de testeo

print('Score del modelo Elastic Net:', elastic_net_m2.score(X_test_m2, y_test_m2))

Score del modelo Elastic Net: 0.2248424985308768


In [33]:
print('Score Elastic Net Precio: ', score_EN)
print('Alpha Elastic Net Precio: ', elastic_net.alpha_)
print('l1_ratio Elastic Net Precio: ', elastic_net.l1_ratio_)
print('Score Elastic Net m2: ', score_EN_m2)
print('Alpha Elastic Net m2: ', elastic_net_m2.alpha_)
print('l1_ratio Elastic Net m2: ', elastic_net_m2.l1_ratio_)

Score Elastic Net Precio:  0.5717193307475611
Alpha Elastic Net Precio:  2944.4070814546635
l1_ratio Elastic Net Precio:  1.0
Score Elastic Net m2:  0.2248424985308768
Alpha Elastic Net m2:  10
l1_ratio Elastic Net m2:  1.0


In [62]:
#Creamos un diccionario para luego mostrar los datos en un dataframe

EN_argentina_precio = {'Modelo': "Elastic Net",
                        'Alcance del modelo': 'Nacional',
                        'Variable Obejtivo': 'Precio_ARS',
                        'R2': score_EN,
                        'Intercepto': elastic_net.intercept_,
                        'Alpha del modelo': elastic_net.alpha_,
                        'l1_ratio': elastic_net.l1_ratio_}

EN_argentina_m2 = { 'Modelo': "Elastic Net",
                    'Alcance del modelo': 'Nacional',                      
                    'Variable Obejtivo': 'Precio_m2',
                    'R2': score_EN_m2,
                    'Intercepto': elastic_net_m2.intercept_,
                    'Alpha del modelo': elastic_net_m2.alpha_,
                    'l1_ratio': elastic_net_m2.l1_ratio_}

# Modelo para Capital

In [65]:
df_cap = df.loc[df['Provincia']=='Capital Federal']


In [66]:
Cap_precio = df_cap.drop(['Precio', 'Precio_Dolares','Precio_m2_Dls', 'Precio_m2'], axis = 1)

Cap_m2 = df_cap.drop(['Precio', 'Precio_Dolares','Precio_m2_Dls', 'Precio_ARS'], axis = 1)

In [67]:
Cap_precio = Cap_precio.dropna(axis =0 , how = 'any')
Cap_m2 = Cap_m2.dropna(axis =0 , how = 'any')

In [68]:
#Creamos variables dummies de las variables categoricas

#Barrio
Cap_precio = pd.concat([Cap_precio, pd.get_dummies(data = Cap_precio['Barrio'], prefix= 'barrio')], axis =1)
Cap_m2 = pd.concat([Cap_m2, pd.get_dummies(data = Cap_m2['Barrio'], prefix= 'barrio')], axis = 1)

#Tipo Propiedad
Cap_precio = pd.concat([Cap_precio, pd.get_dummies(data = Cap_precio['Tipo_Propiedad'], prefix= 'tipo_prop')], axis =1)
Cap_m2 = pd.concat([Cap_m2, pd.get_dummies(data = Cap_m2['Tipo_Propiedad'], prefix= 'tipo_prop')], axis = 1)

In [69]:
#Dropeo de las columnas originales
Cap_precio = Cap_precio.drop(['Provincia','Barrio','Tipo_Propiedad'], axis = 1)
Cap_m2 = Cap_m2.drop(['Provincia','Barrio','Tipo_Propiedad'], axis = 1)

In [71]:
#Separamos nuestra matriz de features

X_Cap = Cap_precio.drop(['Precio_ARS'], axis = 1)
y_Cap = Cap_precio['Precio_ARS']

#### Separamos los datos en sets de testeo y training

In [72]:
#Separamos en train y test
X_train, X_test, y_train, y_test = train_test_split(X_Cap, y_Cap, test_size=0.10, random_state=1)

### Regresion Lineal

In [73]:
#Instanciamos el modelo

lm_Cap = linear_model.LinearRegression()

#Entrenamos el modelo
reg_lineal_Cap = lm_Cap.fit(X_train, y_train)

#Guardamos el score del modelo en una variable para poder comparar
R2_reg_lin_precio_Cap = reg_lineal_Cap.score(X_test, y_test)

#Prediccion con el set de testeo

print('Score del Regresion Lineal:', reg_lineal_Cap.score(X_test, y_test))

Score del Regresion Lineal: 0.5996776950274219


#### Como predice el mismo modelo el precio por metro cuadrado?

In [74]:
#Separamos nuestra matriz de features

X_Cap_m2 = Cap_m2.drop(['Precio_m2'], axis = 1)
y_Cap_m2 = Cap_m2['Precio_m2']

In [75]:
#Separamos en train y test
X_train_cap_m2, X_test_cap_m2, y_train_cap_m2, y_test_cap_m2 = train_test_split(X_Cap_m2, y_Cap_m2, test_size=0.10, random_state=0)

In [80]:
#Instanciamos el modelo

lm_Cap_m2 = linear_model.LinearRegression()

#Entrenamos el modelo
reg_lineal_Cap_m2 = lm_Cap_m2.fit(X_train_cap_m2, y_train_cap_m2)

#Guardamos el score del modelo en una variable para poder comparar
R2_reg_lin_Cap_m2 = reg_lineal_Cap_m2.score(X_test_cap_m2, y_test_cap_m2)

#Prediccion con el set de testeo

print('Score del Regresion Lineal:', reg_lineal_Cap_m2.score(X_test_cap_m2, y_test_cap_m2))

Score del Regresion Lineal: 0.05973424789070447


In [79]:
#Creamos un diccionario con la informacion para luego amar un dataframe y comparar los modelos

rl_capital_precio = { 'Modelo': "Regresion Lineal",
                        'Alcance del modelo': 'Capital Federal',
                        'Variable Obejtivo': 'Precio_ARS',
                        'R2': R2_reg_lin_precio_Cap,
                        'Intercepto': reg_lineal_Cap.intercept_}

rl_capital_m2 = { 'Modelo': "Regresion Lineal",
                    'Alcance del modelo': 'Capital Federal',
                    'Variable Obejtivo': 'Precio_m2',
                    'R2': R2_reg_lin_Cap_m2,
                    'Intercepto': reg_lineal_Cap_m2.intercept_}

## Regresion Lineal Ridge para Capital

Recordemos que nuestros sets de training y testeo quedaron definidos de la siguiente manera:

`X_train, X_test, y_train, y_test`: Para el modelar el Precio_ARS

`X_train_cap_m2, X_test_cap_m2, y_train_cap_m2, y_test_cap_m2`: Para modelar el Precio_m2

In [84]:
#Normalizamos X_train
scaler_precio_cap = StandardScaler()
X_train = scaler_precio_cap.fit_transform(X_train)

#Chequeamos la media y el desvio
print('media:',np.mean(X_train[:,1]))
print('desvio:',np.std(X_train[:,1]))

media: 4.980439736636216e-17
desvio: 1.0


In [85]:
#Instanciamos el modelo
lm_ridge_cap_precio =  linear_model.RidgeCV(alphas=[0.1, 20, 100], cv=5)

#Entrenamos el modelo
reg_lineal_ridge_cap_precio = lm_ridge_cap_precio.fit(X_train, y_train)

# Lo utilizamos para predecir en test
X_test = scaler_precio_cap.transform(X_test)

#Guardamos el score en una variable
score_ridge_cap_precio = reg_lineal_ridge_cap_precio.score(X_test, y_test)

#Prediccion con el set de testeo
print('Score del modelo Ridge:', reg_lineal_ridge_cap_precio.score(X_test, y_test))

Score del modelo Ridge: 0.5996412292918982


#### Como predice el mismo modelo el precio por metro cuadrado?

In [86]:
#Normalizamos las features

scaler_cap_m2 = StandardScaler()
X_train_cap_m2 = scaler_cap_m2.fit_transform(X_train_cap_m2)

#Chequeamos la media y el desvio
print('media:',np.mean(X_train_cap_m2[:,1]))
print('desvio:',np.std(X_train_cap_m2[:,1]))

In [89]:
#Instanciamos el modelo
lm_ridge_cap_m2 =  linear_model.RidgeCV(alphas=[0.1, 20, 10], cv=5)

#Entrenamos el modelo
reg_lineal_ridge_cap_m2 = lm_ridge_cap_m2.fit(X_train_cap_m2, y_train_cap_m2)

# Lo utilizamos para predecir en test
X_test_cap_m2 = scaler_cap_m2.transform(X_test_cap_m2)

score_ridge_cap_m2 = reg_lineal_ridge_cap_m2.score(X_test_cap_m2, y_test_cap_m2)

#Prediccion con el set de testeo
print('Score del modelo Ridge:', reg_lineal_ridge_cap_m2.score(X_test_cap_m2, y_test_cap_m2))

Score del modelo Ridge: -2.673209528035471


In [51]:
print('Score Ridge Precio: ', score_ridge_cap_precio)
print('Alpha Ridge Precio: ', reg_lineal_ridge_cap_precio.alpha_)
print('Score Ridge m2: ', score_ridge_cap_m2)
print('Alpha Ridge m2: ', reg_lineal_ridge_cap_m2.alpha_)

Score Ridge Precio:  0.5996412292918984
Alpha Ridge Precio:  20
Score Ridge m2:  0.22313166695196207
Alpha Ridge m2:  100


## Regresion Lasso para Capital

In [53]:
#Instanciamos el modelo
lm_lasso =  linear_model.LassoCV(alphas=[10000, 100000, 1000], cv = 5)

#Entrenamos el modelo
reg_lineal_lasso = lm_lasso.fit(X_train, y_train)

#Guardamos el score
score_lasso_precio = reg_lineal_lasso.score(X_test, y_test)

print('Score del modelo Lasso:', reg_lineal_lasso.score(X_test, y_test))

Score del modelo Lasso: 0.5997338550921824


In [54]:
#Instanciamos el modelo
lm_lasso_m2 =  linear_model.LassoCV(alphas=[10000, 100000, 1000], cv = 5)

#Entrenamos el modelo
reg_lineal_lasso_m2 = lm_lasso_m2.fit(X_train_m2, y_train_m2)

#Guardamos el score
score_lasso_m2 = reg_lineal_lasso_m2.score(X_test_m2, y_test_m2)

print('Score del modelo Lasso:', reg_lineal_lasso_m2.score(X_test_m2, y_test_m2))

Score del modelo Lasso: 0.1321714542257224


In [55]:
print('Score Lasso Precio: ', score_lasso_precio)
print('Alpha Lasso Precio: ', reg_lineal_lasso.alpha_)
print('Score Lasso m2: ', score_lasso_m2)
print('Alpha Lasso m2: ', reg_lineal_lasso_m2.alpha_)

Score Lasso Precio:  0.5997338550921824
Alpha Lasso Precio:  1000
Score Lasso m2:  0.1321714542257224
Alpha Lasso m2:  1000


## Elastic Net para Capital

In [56]:
#Instanciamos el modelo

lm_elastic_net =  linear_model.ElasticNetCV(l1_ratio= [.1, .5, .7, .9, .95, .99, 1],cv = 5)

#Entrenamos el modelo
elastic_net = lm_elastic_net.fit(X_train, y_train)

#Guardamos el score
score_EN = elastic_net.score(X_test, y_test)

#Prediccion con el set de testeo

print('Score del modelo Elastic Net:', elastic_net.score(X_test, y_test))

Score del modelo Elastic Net: 0.59977762825399


In [57]:
#Instanciamos el modelo

lm_elastic_net_m2 =  linear_model.ElasticNetCV(l1_ratio= [.1, .5, .7, .9, .95, .99, 1], alphas = [10, 100, 10],cv = 5)

#Entrenamos el modelo
elastic_net_m2 = lm_elastic_net_m2.fit(X_train_m2, y_train_m2)

#Guardamos el score
score_EN_m2 = elastic_net_m2.score(X_test_m2, y_test_m2)

#Prediccion con el set de testeo

print('Score del modelo Elastic Net:', elastic_net_m2.score(X_test_m2, y_test_m2))

Score del modelo Elastic Net: 0.2248424985308768


In [58]:
print('Score Elastic Net Precio: ', score_EN)
print('Alpha Elastic Net Precio: ', elastic_net.alpha_)
print('l1_ratio Elastic Net Precio: ', elastic_net.l1_ratio_)
print('Score Elastic Net m2: ', score_EN_m2)
print('Alpha Elastic Net m2: ', elastic_net_m2.alpha_)
print('l1_ratio Elastic Net m2: ', elastic_net_m2.l1_ratio_)

Score Elastic Net Precio:  0.59977762825399
Alpha Elastic Net Precio:  2377.3150733866887
l1_ratio Elastic Net Precio:  1.0
Score Elastic Net m2:  0.2248424985308768
Alpha Elastic Net m2:  10
l1_ratio Elastic Net m2:  1.0


In [ ]:
#Analizar estos valores, tiene sentido que de 1.0 el factor optimo? Alpha Elastic Net Precio = 2377 ??

## Modelos para Zona Norte

In [59]:
df_zona_norte = df.loc[df['Provincia']=='Bs.As. G.B.A. Zona Norte']

zona_norte_precio = df_zona_norte.drop(['Precio', 'Precio_Dolares','Precio_m2_Dls', 'Precio_m2'], axis = 1)

zona_norte_m2 = df_zona_norte.drop(['Precio', 'Precio_Dolares','Precio_m2_Dls', 'Precio_ARS'], axis = 1)

zona_norte_precio = zona_norte_precio.dropna(axis =0 , how = 'any')
zona_norte_m2 = zona_norte_m2.dropna(axis =0 , how = 'any')

#Creamos variables dummies de las variables categoricas

#Barrio
zona_norte_precio = pd.concat([zona_norte_precio, pd.get_dummies(data = zona_norte_precio['Barrio'], prefix= 'barrio')], axis =1)
zona_norte_m2 = pd.concat([zona_norte_m2, pd.get_dummies(data = zona_norte_m2['Barrio'], prefix= 'barrio')], axis = 1)

#Tipo Propiedad
zona_norte_precio = pd.concat([zona_norte_precio, pd.get_dummies(data = zona_norte_precio['Tipo_Propiedad'], prefix= 'tipo_prop')], axis =1)
zona_norte_m2 = pd.concat([zona_norte_m2, pd.get_dummies(data = zona_norte_m2['Tipo_Propiedad'], prefix= 'tipo_prop')], axis = 1)

#Dropeo de las columnas originales
zona_norte_precio = zona_norte_precio.drop(['Provincia','Barrio','Tipo_Propiedad'], axis = 1)
zona_norte_m2 = zona_norte_m2.drop(['Provincia','Barrio','Tipo_Propiedad'], axis = 1)

#Separamos nuestra matriz de features

X_zona_norte = zona_norte_precio.drop(['Precio_ARS'], axis = 1)
y_zona_norte = zona_norte_precio['Precio_ARS']


### Regresión Lineal para Zona Norte

In [60]:
#Separamos en train y test
X_train, X_test, y_train, y_test = train_test_split(X_zona_norte, y_zona_norte, test_size=0.10, random_state=1)

In [61]:
#Instanciamos el modelo

lm_zona_norte = linear_model.LinearRegression()

#Entrenamos el modelo
reg_lineal_zona_norte = lm_zona_norte.fit(X_train, y_train)

#Guardamos el score del modelo en una variable para poder comparar
R2_reg_lin_precio_zona_norte = reg_lineal_zona_norte.score(X_test, y_test)

#Prediccion con el set de testeo

print('Score del Regresion Lineal:', reg_lineal_zona_norte.score(X_test, y_test))

Score del Regresion Lineal: 0.6269661295824513


In [62]:
#Separamos nuestra matriz de features

X_zona_norte_m2 = zona_norte_m2.drop(['Precio_m2'], axis = 1)
y_zona_norte_m2 = zona_norte_m2['Precio_m2']

#Separamos en train y test
X_train_m2, X_test_m2, y_train_m2, y_test_m2 = train_test_split(X_zona_norte_m2, y_zona_norte_m2, test_size=0.10, random_state=0)

In [69]:
#Instanciamos el modelo

lm_zona_norte_m2 = linear_model.LinearRegression()

#Entrenamos el modelo
reg_lineal_zona_norte_m2 = lm_zona_norte_m2.fit(X_train_m2, y_train_m2)

#Guardamos el score del modelo en una variable para poder comparar
R2_reg_lin_zona_norte_m2 = reg_lineal_zona_norte_m2.score(X_test_m2, y_test_m2)

#Prediccion con el set de testeo

print('Score del Regresion Lineal:', reg_lineal_zona_norte_m2.score(X_test_m2, y_test_m2))

Score del Regresion Lineal: -8.706162793580805e+24


In [ ]:
#algo dió mal ahi en el score de zona norte m2.Chequear el codigo si no copie mal o correrlo de vuelta de cero.

### Regresión Ridge para Zona Norte

In [64]:
#normalizamos X_train

X_train = scaler_precio.fit_transform(X_train)

#Instanciamos el modelo

lm_ridge =  linear_model.RidgeCV(alphas=[0.1, 20, 100], cv=5)

#Entrenamos el modelo
reg_lineal_ridge = lm_ridge.fit(X_train, y_train)

# Lo utilizamos para predecir en test
X_test = scaler_precio.transform(X_test)

#Guardamos el score en una variable
score_ridge_precio = reg_lineal_ridge.score(X_test, y_test)

#Prediccion con el set de testeo

print('Score del modelo Ridge:', reg_lineal_ridge.score(X_test, y_test))

Score del modelo Ridge: 0.6278825223874769


In [70]:
#Normalizamos las features

X_train_zona_m2 = scaler_m2.fit_transform(X_train_zona_norte_m2)

#Instanciamos el modelo
lm_ridge_m2 =  linear_model.RidgeCV(alphas=[0.1, 20, 100], cv=5)

#Entrenamos el modelo
reg_lineal_ridge_m2 = lm_ridge_m2.fit(X_train_zona_norte_m2, y_train_zona_norte_m2)

# Lo utilizamos para predecir en test
X_test_zona_norte_m2 = scaler_m2.transform(X_test_zona_norte_m2)

score_ridge_m2 = reg_lineal_ridge_m2.score(X_test_zona_norte_m2, y_test_zona_norte_m2)

#Prediccion con el set de testeo
print('Score del modelo Ridge:', reg_lineal_ridge_m2.score(X_test_zona_norte_m2, y_test_zona_norte_m2))

Score del modelo Ridge: -301.69130206727834
